In [46]:
import json

commodities_raw = json.load(open('all_commodities.json','r'))

FileNotFoundError: [Errno 2] No such file or directory: 'all_commodities.json'

In [ ]:
# the json data received at:
    # endpoint: https://digitaalerfgoed.poolparty.biz/PoolParty/sparql/globalise
    # query:

        '''
        PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
        SELECT DISTINCT ?Concept ?prefLabel ?altLabel
        WHERE
        { ?Concept ?x skos:Concept ; skos:inScheme <https://digitaalerfgoed.poolparty.biz/globalise/be873c02-658f-4764-a010-f00840f7f087> .
        { ?Concept skos:prefLabel ?prefLabel . }
        OPTIONAL { ?Concept skos:altLabel ?altLabel} #FILTER(LANG(?altLabel) = 'nl') }
        } GROUP BY ?Concept ?prefLabel ?altLabel
        '''

In [53]:
len(commodities_raw['results']['bindings'])

4615

In [54]:
commodities_raw['results']['bindings']

[{'Concept': {'type': 'uri',
   'value': 'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab'},
  'prefLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Food and live animals'}},
 {'Concept': {'type': 'uri',
   'value': 'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab'},
  'prefLabel': {'xml:lang': 'nl',
   'type': 'literal',
   'value': 'Voedsel en levende dieren'}},
 {'Concept': {'type': 'uri',
   'value': 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382'},
  'prefLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': "Dairy products and birds' eggs"}},
 {'Concept': {'type': 'uri',
   'value': 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382'},
  'prefLabel': {'xml:lang': 'nl',
   'type': 'literal',
   'value': 'Zuivelproducten en vogeleieren'}},
 {'Concept': {'type': 'uri',
   'value': 'https://digitaalerfgoed.poolparty.bi

In [56]:
language_per_label = {}

for item in commodities_raw['results']['bindings']:
    prefLabel = item['prefLabel']['value']
    lang = item['prefLabel']['xml:lang']
    language_per_label[prefLabel] = lang
    if 'altLabel' in item:
        altLabel = item['altLabel']['value']
        lang = item['altLabel']['xml:lang']
        language_per_label[altLabel] = lang


In [57]:
records = [{'concept_uri':item['Concept']['value'],
           'prefLabel':item['prefLabel']['value'],
           'altLabel':item['altLabel']['value'] if 'altLabel' in item else ''}
            for item in commodities_raw['results']['bindings']
#                 if item['prefLabel']['xml:lang']=='nl'  # Dutch only
          ] # a nicer format, preparing for pandas
records

[{'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab',
  'prefLabel': 'Food and live animals',
  'altLabel': ''},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab',
  'prefLabel': 'Voedsel en levende dieren',
  'altLabel': ''},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382',
  'prefLabel': "Dairy products and birds' eggs",
  'altLabel': ''},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382',
  'prefLabel': 'Zuivelproducten en vogeleieren',
  'altLabel': ''},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/f9dd952e-f882-4929-91a7-a34f7d83e633',
  'prefLabel': 'Zuivel',
  'altLabel': ''},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/f9dd952e-f882-4929-91a7-a34f7d83e633',
  'prefLabel': 'Dairy',
  'altLabel': ''},
 {'concept_uri': 'https:/

In [58]:
labels_by_uri = {}


for record in records:
    concept_uri, prefLabel, altLabel = record['concept_uri'], record['prefLabel'], record['altLabel']
    if concept_uri in labels_by_uri:
        labels_by_uri[concept_uri].add(prefLabel)
    else:
        labels_by_uri[concept_uri] = set([prefLabel])
        
    if altLabel:
        labels_by_uri[concept_uri].add(altLabel)

In [59]:
labels_by_uri

{'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab': {'Food and live animals',
  'Voedsel en levende dieren'},
 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382': {"Dairy products and birds' eggs",
  'Zuivelproducten en vogeleieren'},
 'https://digitaalerfgoed.poolparty.biz/globalise/f9dd952e-f882-4929-91a7-a34f7d83e633': {'Dairy',
  'Zuivel'},
 'https://digitaalerfgoed.poolparty.biz/globalise/4087e492-2e06-427b-ad16-6297e0eea240': {'Boter',
  'Butter'},
 'https://digitaalerfgoed.poolparty.biz/globalise/b4bb6c13-4605-428d-96e9-b526a8ddf830': {'Levende dieren buiten de categorie zeedieren',
  'Live animals other than the marine category'},
 'https://digitaalerfgoed.poolparty.biz/globalise/48917f08-b48a-43d2-8ecc-d2c5eb5636f1': {'Mammals',
  'Zoogdier'},
 'https://digitaalerfgoed.poolparty.biz/globalise/23d0d70b-218e-4934-8fae-94d33812d84f': {'Hengst',
  'Horse',
  'Merrie',
  'Paard',
  'Ruin',
  'Veulen'},
 'https

In [60]:
labels_by_uri_clean = [{'concept_uri':key,
                        'labels': "; ".join(list(labels_by_uri[key]))
                       }
                        for key in labels_by_uri.keys()
                      ]

In [61]:
labels_by_uri_clean

[{'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/e990e5db-88f1-49ab-b49c-609644eed6ab',
  'labels': 'Voedsel en levende dieren; Food and live animals'},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/9c5f5586-a433-49bd-ae6c-e48824e3c382',
  'labels': "Dairy products and birds' eggs; Zuivelproducten en vogeleieren"},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/f9dd952e-f882-4929-91a7-a34f7d83e633',
  'labels': 'Dairy; Zuivel'},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/4087e492-2e06-427b-ad16-6297e0eea240',
  'labels': 'Butter; Boter'},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/b4bb6c13-4605-428d-96e9-b526a8ddf830',
  'labels': 'Live animals other than the marine category; Levende dieren buiten de categorie zeedieren'},
 {'concept_uri': 'https://digitaalerfgoed.poolparty.biz/globalise/48917f08-b48a-43d2-8ecc-d2c5eb5636f1',
  'labels': 'Zoogdier; Mammals'},
 {'concept_uri': 'https://

In [62]:
import pandas as pd

In [63]:
df = pd.DataFrame.from_records(labels_by_uri_clean, columns=['concept_uri','labels'])

In [64]:
df

,concept_uri,labels
0,https://digitaalerfgoed.poolparty.biz/globalis...,Voedsel en levende dieren; Food and live animals
1,https://digitaalerfgoed.poolparty.biz/globalis...,Dairy products and birds' eggs; Zuivelproducte...
2,https://digitaalerfgoed.poolparty.biz/globalis...,Dairy; Zuivel
3,https://digitaalerfgoed.poolparty.biz/globalis...,Butter; Boter
4,https://digitaalerfgoed.poolparty.biz/globalis...,Live animals other than the marine category; L...
...,...,...
3616,https://digitaalerfgoed.poolparty.biz/globalis...,zonnescherm
3617,https://digitaalerfgoed.poolparty.biz/globalis...,zool
3618,https://digitaalerfgoed.poolparty.biz/globalis...,zoopjesglas
3619,https://digitaalerfgoed.poolparty.biz/globalis...,zoopjeskelk


In [65]:
df.to_csv('commodities.tsv',sep='\t')

In [66]:
preferred_label_by_uri = {} 


for record in records:
    concept_uri, prefLabel = record['concept_uri'], record['prefLabel']
    if language_per_label[prefLabel] == 'nl' or concept_uri not in preferred_label_by_uri: 
        # either the Dutch label or the only label - eg. indigo
        preferred_label_by_uri[concept_uri] = prefLabel

In [67]:
json.dump(preferred_label_by_uri,open('preferred_label_by_uri.json','w'))

In [68]:
### Now, let's save broad labels to categorise the commodities

In [69]:
broad_labels_raw = json.load(open('all_broad_labels.json','r'))

In [ ]:
# the json data received at:
    # endpoint: https://digitaalerfgoed.poolparty.biz/PoolParty/sparql/globalise
    # query:
    '''
    PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
    SELECT ?concept ?prefLabel ?broadLabel ?parent
    WHERE
    {
     ?concept skos:prefLabel ?prefLabel ;
        skos:broader ?parent .
     ?parent skos:prefLabel ?broadLabel .
    }
    '''

In [83]:

broad_label_by_uri = {}
for item in broad_labels_raw['results']['bindings']:
    uri = item['concept']['value']
    lang = item['broadLabel']['xml:lang']
    label = item['broadLabel']['value']
    if lang == 'nl' or uri not in broad_label_by_uri:
        broad_label_by_uri[uri] = label

In [84]:
json.dump(broad_label_by_uri, open('broad_label_by_uri.json','w'))

In [85]:
broad_label_by_name = {}
for item in broad_labels_raw['results']['bindings']:
    name = item['prefLabel']['value']
    lang = item['broadLabel']['xml:lang']
    label = item['broadLabel']['value']
    if lang == 'nl' or name not in broad_label_by_name:
        broad_label_by_name[name] = label

In [86]:
json.dump(broad_label_by_name, open('broad_label_by_name.json','w'))

In [91]:
broad_label_by_name['Kruidnagel']

'Specerijen'

In [92]:
broad_label_by_uri['https://digitaalerfgoed.poolparty.biz/globalise/511e4b40-26a7-4da2-b758-a57699c697d0']

'Machines en transportmiddelen'

In [ ]:
tree_of_uris = {}
for item in broad_labels_raw['results']['bindings']:
    name = item['prefLabel']['value']
    label = item['broadLabel']['value']
    